# Create NCSBI 2013-2014 Raw Datasets by County
* This program downloads all original datasets from http://crimereporting.ncsbi.gov/Reports.aspx and saves them as .csv files. These data files are used to create all the flattened and machine learning datasets
    1. This program will dowload specific year data from each web page
    2. It will loop the though 100 counties to get related year data
    3. We generate a new csv file and save specific year data for all of the NC counties

In [44]:
# import libaraies
import requests
import urllib.request
import time
import csv
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from urllib.request import urlopen

In [2]:
year = 2014
# root URL for the crime rate in 2017 including 10 years dataset for each county, but we only want to e
rootPath = 'http://crimereporting.ncsbi.gov/public/2017/CrimeTrds/IndexRateCoTrd/report1/'


In [3]:
# Get the header

url = 'http://crimereporting.ncsbi.gov/public/2017/CrimeTrds/IndexRateCoTrd/report1/1.htm'
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")
# fourth table has the header
table = soup.findAll('table')[4]
headers = []
header = []
header.append('County')

# header is in the first row
for val in table.find_all('tr')[0].find_all('td'):
    #print(val.text)
    header.append(val.text)
headers.append(header)

In [4]:
def getCrimeDataByYear(url,yearToExtract):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    
    title = soup.findAll('table')[3]
    county = title.findAll('td')[0].text + ' County'
    
    table = soup.findAll('table')[4]
   
    for row in table.find_all('tr')[1:]:

        data =[]
        data.append(county)
        firstCol = True
        for val in row.find_all('td'):
            if(firstCol==True):
                yearVal = int(val.text)
            fieldVal = val.text
            if(fieldVal == '\xa0'):
                fieldVal = ''
            data.append(fieldVal)
            firstCol=False
        #print(yearVal,year)

        if(yearVal==yearToExtract):
            #print('true')
            rows.append(data)
    
    #print(headers)
    #print(rows)

In [149]:
# Loop for all of the counties to get the data for specific year
crime =pd.DataFrame([])
for i in range(1,100):
    url = rootPath+str(i)+'.htm'
    getCrimeDataByYear(url,year)


#rows
#crime = pd.DataFrame(np.array(rows).reshape(-1,13), columns = list(headers))
crime.columns = list(headers)


ValueError: Length mismatch: Expected axis has 0 elements, new values have 13 elements

In [148]:
crime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 13 columns):
(County,)           0 non-null float64
(Year,)             0 non-null float64
(Index Rate,)       0 non-null float64
(Violent Rate,)     0 non-null float64
(Property Rate,)    0 non-null float64
(Murder Rate,)      0 non-null float64
(Rape Rate,)        0 non-null float64
(Robbery Rate,)     0 non-null float64
(Assault Rate,)     0 non-null float64
(Burglary Rate,)    0 non-null float64
(Larceny Rate,)     0 non-null float64
(MVT Rate,)         0 non-null float64
(Arson Rate,)       0 non-null float64
dtypes: float64(13)
memory usage: 76.0 bytes


In [11]:
inputFile = '../../Raw Datasets/profile.csv'
profile = pd.read_csv(inputFile, low_memory=False, dtype={'unit_code': object})

In [98]:
lea = profile[['unit_code','School_Name','Lea_Name']].drop_duplicates()
lea['County'] = lea['Lea_Name'].str.replace(r'Schools', '')
lea = lea.sort_values(by=['County'])


In [100]:
lea

,unit_code,School_Name,Lea_Name,County
0,010303,Alamance-Burlington Middle/Early College,Alamance-Burlington Schools,Alamance-Burlington
21,010362,Marvin B Smith Elementary,Alamance-Burlington Schools,Alamance-Burlington
22,010364,North Graham Elementary,Alamance-Burlington Schools,Alamance-Burlington
23,010372,"Pleasant Grove Elementary, Burlington",Alamance-Burlington Schools,Alamance-Burlington
24,010374,R Homer Andrews Elementary,Alamance-Burlington Schools,Alamance-Burlington
25,010378,Ray Street Academy,Alamance-Burlington Schools,Alamance-Burlington
26,010380,South Graham Elementary,Alamance-Burlington Schools,Alamance-Burlington
27,010384,South Mebane Elementary,Alamance-Burlington Schools,Alamance-Burlington
28,010388,Southern High,Alamance-Burlington Schools,Alamance-Burlington
30,010392,Sylvan Elementary,Alamance-Burlington Schools,Alamance-Burlington


In [126]:
crime = pd.DataFrame(crime)

In [144]:
lea['County'].unique()

array(['Alamance-Burlington ', 'Alexander County ', 'Alleghany County ',
       'Anson County ', 'Ashe County ', 'Asheboro City ',
       'Asheville City ', 'Avery County ', 'Beaufort County ',
       'Bertie County ', 'Bladen County ', 'Brunswick County ',
       'Buncombe County ', 'Burke County ', 'Cabarrus County ',
       'Caldwell County ', 'Camden County ', 'Carteret County Public ',
       'Caswell County ', 'Catawba County ', 'Chapel Hill-Carrboro ',
       'Charlotte-Mecklenburg ', 'Charter and Non-District Affiliated ',
       'Chatham County ', 'Cherokee County ', 'Clay County ',
       'Cleveland County ', 'Clinton City ', 'Columbus County ',
       'Craven County ', 'Cumberland County ', 'Currituck County ',
       'Dare County ', 'Davidson County ', 'Davie County ',
       'Duplin County ', 'Durham Public ', 'Edenton-Chowan ',
       'Edgecombe County Public School', 'Elkin City ', 'Forsyth County ',
       'Franklin County ', 'Gaston County ', 'Gates County ',
       'G

In [111]:
lea.reset_index(level=0, inplace=True)
crime.reset_index(level=0, inplace=True)

In [112]:
pd.merge(lea , crime, on = 'County', how = 'left')

ValueError: The column label 'County' is not unique.
For a multi-index, the label must be a tuple with elements corresponding to each level.

In [6]:
#credit: https://www.programiz.com/python-programming/working-csv-files

csv.register_dialect('myDialect',
delimiter = ',',
quotechar = '"',
quoting=csv.QUOTE_ALL,
skipinitialspace=True)
# write the data to csv file
with open('../CrimeData2014.csv', 'w',newline='\n', encoding='utf8') as file:
    writer = csv.writer(file, dialect="myDialect")
    writer.writerows(header for header in headers if header)
    writer.writerows(row for row in rows if row)
file.close()